# Ajuste de hiper-parámetros en regresión

**Alumno:** VASQUEZ RAMOS, Jose Manuel

**Fecha:** 20/05/2025

___

## Contexto académico & objetivo

Ajuste de hiper-parámetros en regresión sobre dataset HousePricesUNS.csv (precio de alquiler universitario). Implementa hill climbing + población usando DEAP para optimizar (α,ridge) de un modelo Ridge. Métrica: RMSE.

## Indicaciones clave

1) Usa deap (creator, toolbox, algorithms) con población = 20.
2) Operadores: mutación gaussiana pequeña, sin cruce.
3) Selección greedily (mejor vecino).
4) Reporta α óptimo y curva de convergencia.

## Librerías

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from deap import base, creator, tools

## Control de aleatoriedad

In [2]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

## Dataset

In [3]:
df = pd.read_csv("dataset/housePrices.csv")
X = df[['Rooms', 'Area_m2']].values
y = df['Price_Soles'].values

df.head()

,Rooms,Area_m2,Price_Soles
0,3,105,6390
1,1,26,10940
2,4,68,10080
3,4,60,7560
4,1,46,9990


## Prepración de datos

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

## Función de fitness (minimizar RMSE)

In [5]:
def evaluate(individual):
    alpha = individual[0]
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return (rmse,)

## Configuración de DEAP

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_alpha", random.uniform, 0.01, 100.0)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_alpha, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1.0, indpb=1.0)
toolbox.register("select", tools.selBest)

## Algoritmo de Hill Climbing con población

In [7]:
def hill_climb(pop_size=20, n_generations=50):
    population = toolbox.population(n=pop_size)
    convergence = []

    # Evaluar la población inicial
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(n_generations):
        offspring = []

        # Mutar cada individuo para obtener un vecino
        for ind in population:
            neighbor = toolbox.clone(ind)
            toolbox.mutate(neighbor)
            neighbor[0] = max(0.01, neighbor[0])  # Mantener en rango válido
            neighbor.fitness.values = toolbox.evaluate(neighbor)

            # Selección greedy: mejor entre actual y vecino
            if neighbor.fitness.values[0] < ind.fitness.values[0]:
                offspring.append(neighbor)
            else:
                offspring.append(ind)

        population = offspring
        best_rmse = tools.selBest(population, 1)[0].fitness.values[0]
        convergence.append(best_rmse)
        print(f"Generación {gen}: RMSE = {best_rmse:.4f}")

    best_individual = tools.selBest(population, 1)[0]
    return best_individual, convergence

## Ejecución

In [8]:
best, convergence = hill_climb()

print(f"\nα óptimo encontrado: {best[0]:.4f}")

Generación 0: RMSE = 2214.0527
Generación 1: RMSE = 2214.0470
Generación 2: RMSE = 2214.0470
Generación 3: RMSE = 2214.0470
Generación 4: RMSE = 2214.0470
Generación 5: RMSE = 2214.0470
Generación 6: RMSE = 2214.0470
Generación 7: RMSE = 2214.0470
Generación 8: RMSE = 2214.0470
Generación 9: RMSE = 2214.0470
Generación 10: RMSE = 2214.0470
Generación 11: RMSE = 2214.0470
Generación 12: RMSE = 2214.0470
Generación 13: RMSE = 2214.0470
Generación 14: RMSE = 2214.0470
Generación 15: RMSE = 2214.0470
Generación 16: RMSE = 2214.0470
Generación 17: RMSE = 2214.0470
Generación 18: RMSE = 2214.0470
Generación 19: RMSE = 2214.0470
Generación 20: RMSE = 2214.0470
Generación 21: RMSE = 2214.0470
Generación 22: RMSE = 2214.0470
Generación 23: RMSE = 2214.0470
Generación 24: RMSE = 2214.0470
Generación 25: RMSE = 2214.0470
Generación 26: RMSE = 2214.0470
Generación 27: RMSE = 2214.0470
Generación 28: RMSE = 2214.0470
Generación 29: RMSE = 2214.0470
Generación 30: RMSE = 2214.0470
Generación 31: RMS